#Move to folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Project/squad/qanet

/content/drive/MyDrive/Project/squad/qanet


In [ ]:
import torch
import numpy as np
import pandas as pd
import pickle
import re, os, string, typing, gc, json
import spacy
from collections import Counter

from torch import nn
import torch
import numpy as np
import pandas as pd
import pickle, time
import re, os, string, typing, gc, json
import torch.nn.functional as F
import spacy
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm
nlp = spacy.blank('en')

# import sys
# sys.path.insert(1, '%cd /drive/MyDrive/Project/squad')
from drive.MyDrive.Project.squad.preprocess import *

#Preprocess Data

In [ ]:
train_data = load_json("../data/squad_train.json")
valid_data = load_json("../data/squad_dev.json")

train_lst = parse_data(train_data)
valid_lst = parse_data(valid_data)

print(f"Train Length: {len(train_lst)}")
print(f"Valid Length: {len(valid_lst)}")

Length of data:  442
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  University_of_Notre_Dame
Length of data:  48
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  Super_Bowl_50
a
Congress often writes legislation to restrain executive officials to the performance of their duties, as laid out by the laws Congress passes. In INS v. Chadha (1983), the Supreme Court decided (a) The prescription for legislative action in Art. I, § 1—requiring all legislative powers to be vested in a Congress consisting of a Senate and a House of Representatives—and § 7—requiring every bill passed by the House and Senate, before becoming law, to be presented to the president, and, if he disapproves, to be repassed by two-thirds of the Senate and House—represents the Framers' decision that the legislative power of the Federal Government be exercised in accord with a single, finely wrought and exhaustively considered procedure. This procedure is an integral part of the constitutional design for th

In [ ]:
train_df = pd.DataFrame(train_lst)
valid_df = pd.DataFrame(valid_lst)

In [ ]:
train_df.head()

,id,context,question,label,answer
0,5733be284776f41900661182,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"[515, 541]",Saint Bernadette Soubirous
1,5733be284776f4190066117f,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"[188, 213]",a copper statue of Christ
2,5733be284776f41900661180,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"[279, 296]",the Main Building
3,5733be284776f41900661181,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,"[381, 420]",a Marian place of prayer and reflection
4,5733be284776f4190066117e,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,"[92, 126]",a golden statue of the Virgin Mary


In [ ]:
drop_ids_train = filter_large_examples(train_df)
train_df.drop(list(drop_ids_train), inplace=True)

drop_ids_valid = filter_large_examples(valid_df)
valid_df.drop(list(drop_ids_valid), inplace=True)

In [ ]:
# def preprocess_df(df):
    
#     def to_lower(text):
#         return text.lower()

#     df.context = df.context.apply(to_lower)
#     df.question = df.question.apply(to_lower)
#     df.answer = df.answer.apply(to_lower)

In [ ]:
# preprocess_df(train_df)
# preprocess_df(valid_df)

In [ ]:
vocab_text = gather_text_for_vocab([train_df, valid_df])
print("Number of sentences in dataset: ", len(vocab_text))

Number of sentences in dataset:  118356


In [ ]:
word2idx, idx2word, word_vocab = build_word_vocab(vocab_text)
print("----------------------------------")
char2idx, char_vocab = build_char_vocab(vocab_text)

raw-vocab: 109803
glove-vocab: 109803
vocab-length: 109805
word2idx-length: 109804
----------------------------------
raw-char-vocab: 1400
char-vocab-intersect: 230
char2idx-length: 231


In [ ]:
train_df['context_ids'] = train_df.context.apply(context_to_ids, word2idx=word2idx)
valid_df['context_ids'] = valid_df.context.apply(context_to_ids, word2idx=word2idx)
train_df['question_ids'] = train_df.question.apply(question_to_ids, word2idx=word2idx)
valid_df['question_ids'] = valid_df.question.apply(question_to_ids, word2idx=word2idx)

In [ ]:
train_err = get_error_indices(train_df, idx2word)
valid_err = get_error_indices(valid_df, idx2word)

train_df.drop(train_err, inplace=True)
valid_df.drop(valid_err, inplace=True)

Error indices: 920
Error indices: 359


In [ ]:
train_label_idx = train_df.apply(index_answer, axis=1, idx2word=idx2word)
valid_label_idx = valid_df.apply(index_answer, axis=1, idx2word=idx2word)

train_df['label_idx'] = train_label_idx
valid_df['label_idx'] = valid_label_idx

In [ ]:
len(train_df), len(valid_df)

(86346, 34080)

In [ ]:
train_df.to_pickle('qanettrain.pkl')
valid_df.to_pickle('qanetvalid.pkl')

In [ ]:
import pickle
with open('qanetw2id.pickle','wb') as handle:
    pickle.dump(word2idx, handle)

with open('qanetc2id.pickle','wb') as handle:
    pickle.dump(char2idx, handle)

#Data Loader

In [ ]:
with open('qanetw2id.pickle','rb') as handle:
    word2idx = pickle.load(handle)
with open('qanetc2id.pickle','rb') as handle:
    char2idx = pickle.load(handle)

In [ ]:
train_df = pd.read_pickle('qanettrain.pkl')
valid_df = pd.read_pickle('qanetvalid.pkl')

In [ ]:
idx2word = {v:k for k,v in word2idx.items()}

In [ ]:
class SQuAD:
    def __init__(self, data, batch_size):
        self.batch_size = batch_size
        data = [data[i:i+self.batch_size] for i in range(0, len(data), self.batch_size)]
        self.data = data
        
        
    def __len__(self):
        return len(self.data)
    
    def make_char_vector(self, max_sent_len, sentence, max_word_len=16):
        
        char_vec = torch.zeros(max_sent_len, max_word_len).type(torch.LongTensor)
        
        for i, word in enumerate(nlp(sentence, disable=['parser','tagger','ner'])):
            for j, ch in enumerate(word.text):
                if j == max_word_len:
                    break
                char_vec[i][j] = char2idx.get(ch, 0)
        
        return char_vec     
    
    def get_span(self, text):

        text = nlp(text, disable=['parser','tagger','ner'])
        span = [(w.idx, w.idx+len(w.text)) for w in text]

        return span

    
    def __iter__(self):
        
        for batch in self.data:
            
            spans = []
            ctx_text = []
            answer_text = []
            
             
            for ctx in batch.context:
                ctx_text.append(ctx)
                spans.append(self.get_span(ctx))
            
            for ans in batch.answer:
                answer_text.append(ans)
                
            max_context_len = max([len(ctx) for ctx in batch.context_ids])
            padded_context = torch.LongTensor(len(batch), max_context_len).fill_(1)
            
            for i, ctx in enumerate(batch.context_ids):
                padded_context[i, :len(ctx)] = torch.LongTensor(ctx)
                
            max_word_ctx = 16
          
            char_ctx = torch.zeros(len(batch), max_context_len, max_word_ctx).type(torch.LongTensor)
            for i, context in enumerate(batch.context):
                char_ctx[i] = self.make_char_vector(max_context_len, context)
            
            max_question_len = max([len(ques) for ques in batch.question_ids])
            padded_question = torch.LongTensor(len(batch), max_question_len).fill_(1)
            
            for i, ques in enumerate(batch.question_ids):
                padded_question[i, :len(ques)] = torch.LongTensor(ques)
                
            max_word_ques = 16
            
            char_ques = torch.zeros(len(batch), max_question_len, max_word_ques).type(torch.LongTensor)
            for i, question in enumerate(batch.question):
                char_ques[i] = self.make_char_vector(max_question_len, question)
            
              
            label = torch.LongTensor(list(batch.label_idx))
            ids = list(batch.id)
            
            yield (padded_context, padded_question, char_ctx, char_ques, label, ctx_text, answer_text, ids)
            
         

In [ ]:
train_dataset = SQuAD(train_df, 12)
valid_dataset = SQuAD(valid_df, 12)     

#QANet

##Word Level Embedding

In [ ]:
def get_glove_embedding():
    glove_dict = {}
    with open("../GloVe/glove.840B.300d.txt", "r", encoding="utf-8") as f:
        for line in f:
            lst = line.split(' ')
            word = lst[0]
            vector = np.asarray(lst[1:], dtype="float32")
            glove_dict[word] = vector
    
    f.close()
    return glove_dict


In [ ]:
glove_dict = get_glove_embedding()

In [ ]:
def get_weight_matrix(glove_dict):
    weight_matrix = np.zeros((len(word_vocab), 300))
    num_words = 0
    for i, word in enumerate(word_vocab):
        try:
            weight_matrix[i] = glove_dict[word]
            num_words += 1
        except:
            pass

    return weight_matrix, num_words

In [ ]:
weight_matrix, num_words = get_weight_matrix(glove_dict)
num_words

90777

In [ ]:
np.save("qanetglove.npy", weight_matrix)

##Character Level Embedding (Depthwise Separable Convolutions)

In [ ]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dim=1):
        super(DepthwiseSeparableConv, self).__init__()
        self.dim = dim

        self.depthwise = nn.Conv1d(in_channels, in_channels, kernel_size, padding=kernel_size // 2, groups=in_channels,
                                   bias=False)
        self.pointwise = nn.Conv1d(in_channels, out_channels, 1, bias=True)

        if dim == 2:
            self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size, padding=kernel_size // 2, groups=in_channels,
                                   bias=False)
            self.pointwise = nn.Conv2d(in_channels, out_channels, 1, bias=True)


    def forward(self, x):
        if self.dim == 1:
            x = self.depthwise(x.transpose(1, 2))
            x = self.pointwise(x)
            x = x.transpose(1, 2)
        else:
            x = self.depthwise(x)
            x = self.pointwise(x)

        return x

##Highway Network

In [ ]:
# Highway Network
class Highway(nn.Module):
    def __init__(self, size, num_layers):
        super(Highway, self).__init__()
        self.num_layers = num_layers
        self.nonlinear = nn.ModuleList([nn.Linear(size, size) for _ in range(num_layers)])
        self.linear = nn.ModuleList([nn.Linear(size, size) for _ in range(num_layers)])
        self.gate = nn.ModuleList([nn.Linear(size, size) for _ in range(num_layers)])

    def forward(self, x):
        for i in range(self.num_layers):
            nonlinear = F.relu(self.nonlinear[i](x))
            linear = self.linear[i](x)
            gate = torch.sigmoid(self.gate[i](x))
            x = gate * nonlinear + (1 - gate) * linear
        return x

##Embedding Layer

In [ ]:
# Embedding Layer
class Embedding(nn.Module):
    def __init__(self, char_vocab_size, char_embed_size, char_out_size, device, dropout=0.1):
        super(Embedding, self).__init__()
        self.word_embed_size = None
        self.char_embed_size = char_embed_size
        self.char_out_size = char_out_size
        self.dropout = dropout
        self.device = device

        self.word_embed = self.get_word_embed()
        self.char_embed = nn.Embedding(char_vocab_size, char_embed_size)
        self.char_conv2d = DepthwiseSeparableConv(char_embed_size, char_embed_size, char_out_size, dim=2)

        self.highway = Highway(self.word_embed_size + char_embed_size, 2)

    def get_word_embed(self):
        weights_matrix = np.load('qanetglove.npy')
        num_embeddings, embedding_dim = weights_matrix.shape
        self.word_embed_size = embedding_dim
        word_embed = nn.Embedding.from_pretrained(torch.FloatTensor(weights_matrix), freeze=True).to(self.device)
        return word_embed

    def forward(self, x, char_x):
        x = self.word_embed(x)
        x = F.dropout(x, p=self.dropout)
        char_x = self.char_embed(char_x)

        char_x = F.dropout(char_x.permute(0, 3, 1, 2), p=self.dropout)
        char_x = F.relu(self.char_conv2d(char_x))
        char_x = torch.max(char_x, dim=3)[0]
        char_x = char_x.permute(0, 2, 1)
        x = torch.cat([x, char_x], dim=2)
        x = self.highway(x)
        return x

##Multi-head Attention

In [ ]:
# Multi-Head Self-Attention
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_size, num_heads, dropout=0.1):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_size = embed_size
        self.num_heads = num_heads
        self.head_size = embed_size // num_heads
        self.dropout = dropout

        self.W_Q = nn.Linear(embed_size, embed_size)
        self.W_K = nn.Linear(embed_size, embed_size)
        self.W_V = nn.Linear(embed_size, embed_size)

        self.W_O = nn.Linear(embed_size, embed_size)

    def forward(self, x, mask):
        Q = self.W_Q(x)
        K = self.W_K(x)
        V = self.W_V(x)

        Q = Q.view(-1, Q.size(1), self.num_heads, int(self.embed_size / self.num_heads)).permute(0, 2, 1, 3)
        K = K.view(-1, K.size(1), self.num_heads, int(self.embed_size / self.num_heads)).permute(0, 2, 1, 3)
        V = V.view(-1, V.size(1), self.num_heads, int(self.embed_size / self.num_heads)).permute(0, 2, 1, 3)

        K_T = K.permute(0, 1, 3, 2)
        attention = torch.matmul(Q, K_T) / np.sqrt(self.embed_size / self.num_heads)
        # print(x.size())
        attention = attention - 1e30 * (1 - mask[:, None, None, :])
        attention = F.softmax(attention, dim=-1)
        attention = F.dropout(attention, p=self.dropout)

        x = torch.matmul(attention, V)
        x = x.permute(0, 2, 1, 3).contiguous()
        x = x.view(-1, x.size(1), self.embed_size)
        x = self.W_O(x)
        return x

## Positional Encoder

In [ ]:
# Positional Encoder
class PositionalEncoder(nn.Module):
    def __init__(self, embed_size, device, dropout=0.1):
        super(PositionalEncoder, self).__init__()
        self.dropout = dropout
        self.embed_size = embed_size
        self.device = device
        self.W_P = nn.Linear(embed_size, embed_size)

    def forward(self, x):
        batch_size, seq_len, embed_size = x.size()
        position = torch.arange(seq_len, dtype=torch.float32).to(self.device)
        position = position.unsqueeze(0).expand(batch_size, seq_len)
        position = position.unsqueeze(2).expand(batch_size, seq_len, embed_size)
        # print(position.size())
        position = position / torch.pow(10000, torch.arange(0, embed_size, dtype=torch.float32, step=1).to(self.device) / embed_size)
        position[:, :, 0::2] = torch.sin(position[:, :, 0::2])
        position[:, :, 1::2] = torch.cos(position[:, :, 1::2])
        x = x + position
        x = self.W_P(x)
        x = F.dropout(x, p=self.dropout)
        return x

##Context-Query Attention Layer

In [ ]:
class BiAttention(nn.Module):
    def __init__(self, embed_size, dropout=0.1):
        super(ContextQueryAttention, self).__init__()
        self.dropout = dropout
        self.W_O = nn.Linear(3*embed_size, 1)

    def forward(self, C, Q, C_mask, Q_mask):
        C_len = C_mask.size(1)
        Q_len = Q_mask.size(1)

        C_mask = C_mask[:, :, None]
        Q_mask = Q_mask[:, None, :]


        C_ = C.unsqueeze(2).expand(-1, -1, Q_len, -1)
        Q_ = Q.unsqueeze(1).expand(-1, C_len, -1, -1)
        element_wise_product = C_ * Q_
        S = torch.cat([C_, Q_, element_wise_product], dim=-1)
        S = self.W_O(S).squeeze(-1)
        S = S - 1e30 * (1 - C_mask) - 1e30 * (1 - Q_mask)
        S_C = F.softmax(S, dim=2)
        S_C = F.dropout(S_C, p=self.dropout)
        S_Q = F.softmax(S, dim=1)
        S_Q = F.dropout(S_Q, p=self.dropout)

        C2Q = torch.matmul(S_C, Q)
        Q2C = torch.matmul(torch.matmul(S_C, S_Q.permute(0, 2, 1)), C)

        out = torch.cat([C, C2Q, C * C2Q, C * Q2C], dim=-1)

        return out

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_size, dropout=0.1, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.dropout = dropout
        self.head_size = int(hidden_size / num_heads)
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.dense = nn.Linear(2*hidden_size, hidden_size)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_heads, self.head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states, memory, attention_mask):
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(memory)
        mixed_value_layer = self.value(memory)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.head_size)
        attention_scores = attention_scores - 1e30 * (1 - attention_mask[:, None, None, :])

        attention_probs = nn.Softmax(dim=-1)(attention_scores)
        attention_probs = nn.Dropout(self.dropout)(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.hidden_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        ou = torch.cat([hidden_states, context_layer], dim=-1)

        attention_output = self.dense(ou)
        return F.relu(attention_output)

##Encoder Block

In [ ]:
# Encoder Block
class EncoderBlock(nn.Module):
    def __init__(self, embed_size, num_heads, num_conv_layers, conv_kernel_size, device, dropout=0.1):
        super(EncoderBlock, self).__init__()
        self.embed_size = embed_size
        self.num_conv_layers = num_conv_layers
        self.num_heads = num_heads
        self.conv_kernel_size = conv_kernel_size
        self.dropout = dropout

        self.self_attention = MultiHeadSelfAttention(embed_size, num_heads, dropout)
        self.self_attention_norm = nn.LayerNorm(embed_size)

        self.position_encoder = PositionalEncoder(embed_size, device, dropout)
        self.position_encoder_norm = nn.LayerNorm(embed_size)

        self.conv_layers = nn.ModuleList([DepthwiseSeparableConv(embed_size, embed_size, conv_kernel_size, dim=1) for _ in range(num_conv_layers)])
        self.conv_norm = nn.ModuleList([nn.LayerNorm(embed_size) for _ in range(num_conv_layers)])

        self.feed_forward = nn.Linear(embed_size, embed_size)
        self.feed_forward_norm = nn.LayerNorm(embed_size)

    def forward(self, x, mask):
        position_encoder = self.position_encoder(x)
        residual = x
        x = self.position_encoder_norm(position_encoder)
        for i in range(self.num_conv_layers):
            x = F.relu(self.conv_layers[i](x), inplace=False)
            x += residual
            if (i+1)%2 == 0:
                x = F.dropout(x, p=self.dropout)
            residual = x
            x = self.conv_norm[i](x)

        x = self.self_attention(x, mask)
        x = F.dropout(x+residual, p=self.dropout)
        residual = x
        x = self.self_attention_norm(x)

        x = self.feed_forward(x)
        x = F.dropout(x+residual, p=self.dropout)
        return x



##Ouput Layer

In [ ]:
# Output Layer
class OutputLayer(nn.Module):
    def __init__(self, embed_size, dropout=0.1):
        super(OutputLayer, self).__init__()
        self.dropout = dropout
        self.W_1 = nn.Linear(2*embed_size, 1, bias=False)
        self.W_2 = nn.Linear(2*embed_size, 1, bias=False)

    def forward(self, M1, M2, M3, C_mask):
        pred_start = torch.cat([M1, M2], dim=-1)
        pred_start = self.W_1(pred_start).squeeze(-1)
        pred_start = pred_start - 1e30 * (1 - C_mask)

        pred_end = torch.cat([M1, M3], dim=-1)
        pred_end = self.W_2(pred_end).squeeze(-1)
        pred_end = pred_end - 1e30 * (1 - C_mask)

        return pred_start, pred_end

##Model

In [ ]:
#QANet Model
class QANet(nn.Module):
    def __init__(self, char_vocab_size, char_embed_size, word_embed_size, embed_size, num_heads, conv_kernel_size, device, dropout=0.1):
        super(QANet, self).__init__()
        self.char_embed_size = char_embed_size
        self.embed_size = embed_size
        self.dropout = dropout
        print(char_embed_size)

        self.embedding = Embedding(char_vocab_size, char_embed_size, conv_kernel_size, device, dropout)
        self.ctx_resizer = DepthwiseSeparableConv(char_embed_size+word_embed_size, embed_size, 5)
        self.query_resizer = DepthwiseSeparableConv(char_embed_size+word_embed_size, embed_size, 5)

        self.embedding_encoder = EncoderBlock(embed_size, num_heads, 4, 5, device, dropout)

        self.context_query_attention = ContextQueryAttention(embed_size, dropout)
        self.cqa_resizer = DepthwiseSeparableConv(4*embed_size, embed_size, 5)

        self.model_encoder_layers = nn.ModuleList([EncoderBlock(embed_size, num_heads, 2, 5, device, dropout) for _ in range(7)])
        self.output_layer = OutputLayer(embed_size, dropout)

        self.device = device

    def forward(self, C_word, Q_word, C_char, Q_char):
        C_mask = (C_word > 0).float()
        Q_mask = (Q_word > 0).float()

        Q_d = torch.zeros_like(C_mask)
        Q_d[:, :Q_mask.size(1)] = Q_mask
        Q_mask = Q_d

        C_embed = self.embedding(C_word, C_char)
        Q_embed = self.embedding(Q_word, Q_char)

        C = self.ctx_resizer(C_embed)
        Q = self.query_resizer(C_embed)

        C = self.embedding_encoder(C, C_mask)
        Q = self.embedding_encoder(Q, Q_mask)

        CQA = self.context_query_attention(C, Q, C_mask, Q_mask)
        M1 = self.cqa_resizer(CQA)

        for layer in self.model_encoder_layers:
            M1 = layer(M1, C_mask)

        M2 = M1

        for layer in self.model_encoder_layers:
            M2 = layer(M2, C_mask)

        M3 = M2

        for layer in self.model_encoder_layers:
            M3 = layer(M3, C_mask)

        pred_start, pred_end = self.output_layer(M1, M2, M3, C_mask)
        return pred_start, pred_end

#Training

In [ ]:
# Training
CHAR_VOCAB_SIZE = len(char2idx)
CHAR_EMBED_SIZE = 200
WORD_EMBED_SIZE = 300
KERNEL_SIZE = 5
EMBED_SIZE = 128
NUM_HEADS = 8

device = torch.device('cpu')
model = QANet(CHAR_VOCAB_SIZE, CHAR_EMBED_SIZE, WORD_EMBED_SIZE, EMBED_SIZE, NUM_HEADS, KERNEL_SIZE, device).to(device)

optimizer = torch.optim.Adam(model.parameters(), betas=(0.8, 0.999), eps=1e-7, weight_decay=3e-7)

200


In [ ]:
def train(model, train_dataset):
    print('Training...')
    model.train()
    total_loss = 0
    batch_count = 0
    for batch in tqdm(train_dataset):
        if batch_count%500 == 0:
            print('Batch: {}'.format(batch_count))

        batch_count += 1

        context, question, char_context, char_question, label, context_text, answer, ids = batch

        pred_start, pred_end = model(context, question, char_context, char_question)

        loss = F.cross_entropy(pred_start, label[:,0]) + F.cross_entropy(pred_end, label[:,1])
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    return total_loss/len(train_dataset)

In [ ]:
def validate(model, val_dataset):
    print('Validating...')
    model.eval()
    valid_loss = 0
    batch_count = 0
    predictions = {}
    for batch in tqdm(val_dataset):
        if batch_count%500 == 0:
            print('Batch: {}'.format(batch_count))

        batch_count += 1

        context, question, char_context, char_question, label, context_text, answer, ids = batch
        context, question, char_context, char_question, label = context.to(device), question.to(device), char_context.to(device), char_question.to(device), label.to(device)

        with torch.no_grad():
            pred_start, pred_end = model(context, question, char_context, char_question)

            loss = F.nll_loss(pred_start, label[:,0]) + F.nll_loss(pred_end, label[:,1])
            valid_loss += loss.item()

            batch_size, seq_len = pred_start.shape
            ls = nn.LogSoftmax(dim=-1)
            mask = (torch.ones(seq_len, seq_len)*float('-inf')).to(device).tril(-1).unsqueeze(0).expand(batch_size, -1, -1)
            score = ls(pred_start).unsqueeze(2) + ls(pred_end).unsqueeze(1) + mask
            score, start_idx = score.max(dim=1)
            score, end_idx = score.max(dim=1)
            start_idx = start_idx.gather(1, end_idx.unsqueeze(1)).squeeze(1)

            for i in range(batch_size):
                id = ids[i]
                pred = context[i][start_idx[i]:end_idx[i]+1]
                pred = ' '.join([idx2word[idx.item()] for idx in pred])
                predictions[id] = pred

    em, f1 = evaluate(predictions)
    return valid_loss/len(val_dataset), em, f1

In [ ]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if len(prediction_tokens) == 0 or len(ground_truth_tokens) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(prediction_tokens == ground_truth_tokens)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


def evaluate(predictions):
    with open('../data/squad_dev.json') as f:
        dataset_json = json.load(f)

    dataset = dataset_json['data']
    f1 = exact_match = total = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                total += 1
                if qa['id'] not in predictions:
                    continue
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = predictions[qa['id']]
                exact_match += metric_max_over_ground_truths(exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return exact_match, f1

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
train_losses = []
valid_losses = []
valid_em = []
valid_f1 = []
epochs = 10

for epoch in range(epochs):
    start_time = time.time()

    train_loss = train(model, train_dataset)
    valid_loss, em, f1 = validate(model, valid_dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    valid_em.append(em)
    valid_f1.append(f1)

    print('Epoch: {} | Time: {}m {}s'.format(epoch + 1, epoch_mins, epoch_secs))
    print('\tTrain Loss: {:.3f}'.format(train_loss))
    print('\t Val. Loss: {:.3f} |  Val. EM: {:.2f}% |  Val. F1: {:.2f}%'.format(valid_loss, em, f1))

Training...


  0%|          | 0/7196 [00:00<?, ?it/s]

Batch: 0


/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py:175: UserWarning: Error detected in ReluBackward0. Traceback of forward call that caused the error:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelapp.py", line 612, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/dist-packages/tornado/platform/asyncio.py", line 149, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
    self._run_once()
  File "/usr/lib/python3.7/asyncio/base_events.py", lin

RuntimeError: ignored

In [ ]:
torch.arange(0, 10, dtype=torch.float32, step=2)